In [1]:
import json
import random
from datasets import load_dataset

import warnings
warnings.filterwarnings("ignore")

DATASET = "wikimedia/wikipedia"
SUBSET = "20231101.en"               # default subset
SPLIT = "train"
STREAMING = True

OUTPUT_PATH = "pretrain_wikipedia_en.jsonl"

MAX_SAMPLES = 500_000       # 5L
MAX_CHARS = 512             # strict limit

CHUNK_SIZE = 50_000         # ensures strong global shuffling


def write_chunk(chunk, path):
    """Shuffle chunk & append to disk."""
    random.shuffle(chunk)
    with open(path, "a", encoding="utf-8") as f:
        for item in chunk:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")


def main():
    print(f"🔄 Streaming dataset: {DATASET} ({SUBSET} subset)")

    ds = load_dataset(
        DATASET,
        name=SUBSET,
        split=SPLIT,
        streaming=STREAMING
    )

    buffer = []
    written = 0
    total_chars = 0
    chunk_id = 1

    for sample in ds:
        if written >= MAX_SAMPLES:
            break

        text = (sample.get("text") or "").strip()
        if not text:
            continue

        # strict length
        if len(text) >= MAX_CHARS:
            continue

        total_chars += len(text)

        buffer.append({
            "text": f"<|im_start|>{text}<|im_end|>"
        })
        written += 1

        # chunk flushing
        if len(buffer) >= CHUNK_SIZE:
            print(f"🧹 Writing chunk {chunk_id} ({len(buffer)}) — Total: {written}")
            write_chunk(buffer, OUTPUT_PATH)
            buffer.clear()
            chunk_id += 1

    # final chunk
    if buffer:
        print(f"🧹 Writing final chunk ({len(buffer)})")
        write_chunk(buffer, OUTPUT_PATH)

    # summary
    print("\n🎉 DONE!")
    print(f"📦 Total samples written: {written}")
    print(f"🔢 Estimated tokens: {total_chars / 4:.0f}")  # approx 4 chars/token
    print(f"🔢 Tokens (millions): {total_chars / 4 / 1e6:.3f}M")
    print(f"📄 Saved to: {OUTPUT_PATH}")


if __name__ == "__main__":
    main()

🔄 Streaming dataset: wikimedia/wikipedia (20231101.en subset)


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

🧹 Writing chunk 1 (50000) — Total: 50000
🧹 Writing chunk 2 (50000) — Total: 100000
🧹 Writing chunk 3 (50000) — Total: 150000
🧹 Writing chunk 4 (50000) — Total: 200000
🧹 Writing chunk 5 (50000) — Total: 250000
🧹 Writing chunk 6 (50000) — Total: 300000
🧹 Writing chunk 7 (50000) — Total: 350000
🧹 Writing chunk 8 (50000) — Total: 400000
🧹 Writing chunk 9 (50000) — Total: 450000
🧹 Writing chunk 10 (50000) — Total: 500000

🎉 DONE!
📦 Total samples written: 500000
🔢 Estimated tokens: 36951168
🔢 Tokens (millions): 36.951M
📄 Saved to: pretrain_wikipedia_en.jsonl
